<h1 style="text-align:center;">Update Firmware via HTTP URL</h1>

<div class="alert  alert-block  alert-info" style="border-radius: 20px;">    
    <ul>
        <li><p>This Jupyter Notebook will contain script for updating Firmware via HTTP URL. This code is written in Python and uses the redfish library to perform the required operations.</p></li>
        <li>As you are here let's get it done easily.</li>
        <li>To perform this you need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style="none">python-ilorest-library</a> installed in your machine.</li>
        <li>Please keep your username and password handy</li>
        <li>Do execute each successive cell once to avoid any errors!</li>
    </ul>    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
    <ul>
        <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys</a></li>
        <li>json</li>
        <li>redfish</li> 
    </ul>
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError

from get_resource_directory import get_resource_directory

#This is to take passwords as input securely
import getpass

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 2. This is our update_ilo_firmware function overview, in the end this will update the iLO Firmware.</h3><br>
    
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to do this.<br>

1. The <b>update_ilo_firmware</b> function takes three parameters - <b>_redfishobj</b> representing the connection to the server, <b>fw_url</b> representing the firmware update URL, and <b>tpm_flag</b> representing the TPM override flag.
    <br>
    
2. The function initializes the <b>body</b> and <b>update_service_uri</b> variables to None.
    <br>

3. The <b>get_resource_directory()</b> function is called by passing the <b>_redfishobj</b> as a parameter to retrieve the resource directory of the server. If the resource directory is not available or it is disabled, the relevant URI is found by navigating through the server's API. If the resource directory is available, the relevant URI is found from the resource directory.
    <br>

4. If <b>update_service_uri</b> and <b>fw_url</b> are not None, the <b>update_uri</b> is retrieved from the server using the <b>get()</b> method and by accessing the Actions attribute.
    <br>

5. The <b>body</b> dictionary is updated with the <b>fw_url</b> and <b>tpm_flag</b> if <b>tpm_flag</b> is provided.
    <br>

6. The <b>post()</b> method is called on the <b>update_uri</b> with the <b>body</b> dictionary as a parameter. The response is stored in the <b>resp</b> variable.
    <br>


7. If the HTTP response status code is 400, the iLO extended info error message is printed. If the HTTP response status code is not 200 or 201, an error message is printed. If the HTTP response status code is 200 or 201, a success message is printed along.
    <br>
</div>

In [ ]:
def update_ilo_firmware(_redfishobj, fw_url, tpm_flag):

    body = dict()
    update_service_uri = None

    resource_instances = get_resource_directory(_redfishobj)
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        update_service_uri = _redfishobj.root.obj['UpdateService']['@odata.id']
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#UpdateService.' in instance['@odata.type']:
                update_service_uri = instance['@odata.id']
                break

    if update_service_uri and fw_url:
        update_uri = _redfishobj.get(update_service_uri).obj['Actions']['#UpdateService.SimpleUpdate']['target']
        body["ImageURI"] = fw_url
        if tpm_flag:
            body["TPMOverrideFlag"] = tpm_flag
        resp = _redfishobj.post(update_uri, body)
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, sort_keys=True))
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, sort_keys=True))

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 3. This is Main function overview.</h3>
    <br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to update the firmware.

1. The rest opertions are handled by redfish object which is created by RedfishClient<br>
<br>
    
2. <b>RedfishClient</b> class takes 3 parameters<i> iLO url, Username, Password</i> and returns a redfish object.<br>
    <br>

3. Once you enter everything a redfish obejct is created with the help of RedfishClient class.<br>
    <br>

4. Now we try to Login with the help of <i> REDFISHOBJ</i>, if it is successful you are logged in, But if           Server       is not avaialbe we get an error saying "server unreachable"<br>
    <br>
5. Then we call <b>update_ilo_firmware</b> with our redfish object, FIRMWARE_URL and Trusted Platform Module (TPM) Flag as parameters.<br>    

6. REDFISHOBJ.logout() will take care of logging you out of the ilo.<br><br>
    
7. <b>When prompted for Firmware URL, enter the firmware file path</b>. Example: "http://<url_to_binary_file>"><br><br>
    
8. If a TPM module (Trusted Platform Module / Cryptographic Co-processor) is installed/present onboard then be sure to include this by changing <b>TPM_FLAG</b> variable to True. By default this variable is set to False.<br>
 

 </div>

<div class="alert alert-block alert-warning" style="border-radius: 20px;">
<b> When running on the server locally,</b>use the following values:<br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address, account name and password to send https requests<br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

In [ ]:
if __name__ == "__main__":
    
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Login account: ")
    LOGIN_PASSWORD = getpass.getpass("Enter your password: ")

    # firmware file path and filename
    FIRMWARE_URL = input("Enter Firmware URL")
    
    # If a TPM module (Trusted Platform Module/ Cryptographic Co-processor) is installed/present
    # onboard then be sure to include this.
    
    TPM_FLAG = False
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = True

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    update_ilo_firmware(REDFISHOBJ, FIRMWARE_URL, TPM_FLAG)
    REDFISHOBJ.logout()

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    In case you need help please check the follwoing links for reference:
    <br>
    1. Python-ilorest-library: <a href = "https://github.com/HewlettPackard/python-ilorest-library">Python ilorest library</a><br>
    2. HPE ilorest-api explorer: <a href = "https://ilorestfulapiexplorer.ext.hpe.com/">ilorestful api explorer</a><br>
    3. HPE iLO Redfish Documentation: <a href = "https://hewlettpackard.github.io/ilo-rest-api-docs/ilo6/">HPE iLO Redfish API Documentation</a><br>
    4. REST API: <a href = "https://restfulapi.net/">Restful API</a><br>    
</div>